## Import Library

In [1]:
# utility
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# keras tensorflow wrapper
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications import InceptionV3, Xception
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.metrics import top_k_categorical_accuracy
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.optimizers import Adam

# scikit-learn helper function
from sklearn.utils.class_weight import compute_class_weight

## Helper functions

In [2]:
def top_3_accuracy(true, pred):
    return top_k_categorical_accuracy(true, pred, k=3)

def path_join(dirname, img_paths):
    return [os.path.join(dirname, img_path) for img_path in img_paths]

## 이미지 데이터 전처리 및 generator 생성

In [12]:
TRAIN_PATH = '../training'
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.1)

batch_size = 32
input_shape = (224,224)

generator_train = datagen.flow_from_directory(directory=TRAIN_PATH,
                                              target_size=input_shape,
                                              shuffle=True,
                                              subset="training"
                                              )

generator_validate = datagen.flow_from_directory(directory=TRAIN_PATH,
                                                 target_size=input_shape,
                                                 shuffle=False,
                                                 subset="validation"
                                                 )
steps_train = generator_train.n / batch_size
steps_validate = generator_validate.n / batch_size

cls_train = generator_train.classes
cls_validate = generator_validate.classes

num_classes = generator_train.num_classes

class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train) 

Found 673383 images belonging to 19 classes.
Found 74813 images belonging to 19 classes.


## 모델 정의 및 구축

In [22]:
class Model():
    def __init__(self, name, class_weight, params):
        assert name != '', "Model name needs to be specified"
        self.name = name
        self.trained = False   
        
    def construct_model(self):
        if self.name == 'inceptionv3':
            print('{:=^75}'.format('Downloading {}'.format(self.name)))
            self.base_model = InceptionV3(**params['network_params'])
            print('{:=^75}'.format('Download Complete'))
            
        elif self.name == 'xception':
            print('{:=^75}'.format('Downloading {}'.format(self.name)))
            self.base_model = Xception(**params['network_params'])
            print('{:=^75}'.format('Download Complete'))
            
            
        # 모델 구조  base model -> global average pooling -> dense
        print('{:=^75}'.format('Adding layers'))
        self.model = Sequential()
        self.model.add(self.base_model)
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(params['num_classes'], activation='softmax'))
        print('{:=^75}'.format('Added layers'))
    
        # 지정 경로에 저장
        if not os.path.exists('weight_path/'):
            os.mkdir('weight_path/')
        self.weight_save_path = os.path.join('weight_path/', self.name + "_weights.h5")
        
        print('{:=^75}'.format('Saving weights to {}'.format(self.weight_save_path)))
        self.model.save_weights(self.weight_save_path)
        print('{:=^75}'.format('Saved weights'))
    
    
    # train with feature extraction
    def train(self):
        if self.trained == True:
            self.model.load_weights(self.weight_save_path)
            self.trained = False
        
        # parma validation
        assert params['mode'] in ['fe', 'ft'], "mode must be either 'fe' or 'ft'"
        
        # 레이어 trainable 지정
        # feature extraction
        if params['mode'] == 'fe':
            self.model.layers[0].trainable = False
            
        # finetuning
        elif params['mode'] == 'ft':
            self.model.layers[0].trainable = True
    
            
        # compile the model with designated parameters    
        self.model.compile(optimizer=Adam(lr=params['lr']),
                           loss='categorical_crossentropy',
                           metrics=['categorical_accuracy', top_3_accuracy])
        
        if not os.path.exists(params['log_path']):
            os.mkdir(params['log_path'])
        
        if not os.path.exists(params['cp_path']):
            os.mkdir(params['cp_path'])
        
        # csv logger callback 
        log_path = os.path.join(params['log_path'], self.name + '_' + params['mode'] + '.log')
        csvlog_callback = CSVLogger(log_path)
        
        # checkpoint callback 
        cp_path = os.path.join(params['cp_path'], self.name + '_' + params['mode'] + '-{epoch:04d}-{val_loss:.2f}.h5')
        cp_callback = ModelCheckpoint(cp_path,
                                      mode="max",
                                      save_best_only=True)
        
        print('{:=^75}'.format('training {} with {}'.format(self.name, params['mode'])))
        # actual data fitting
        self.model.fit_generator(generator=generator_train,
                                  epochs=params['epoch'],
                                  class_weight=class_weight,
                                  validation_data=generator_validate,
                                  validation_steps=steps_validate,
                                  callbacks=[cp_callback, csvlog_callback])
        
        # save model once done training    
        if not os.path.exists(params['model_path']):
            os.mkdir(params['model_path'])
            
        model_save_path = os.path.join(params['model_path'], model.name + '_' + params['mode'] + '.h5')
        self.model.save(model_save_path)
        self.trained = True

In [23]:
params = {
    'num_classes': num_classes,
    'log_path': 'log/',
    'cp_path': 'checkpoint/',
    'model_path': 'model/',
    'mode': 'fe',
    'lr': 0.001,
    'epoch': 10,
    'network_params': {
    'include_top' : False, 
    'weights' : 'imagenet', 
    'input_shape' : input_shape + (3,)
    }
}

In [24]:
inception = Model(name='inceptionv3', class_weight=class_weight, params=params)
xception = Model(name='xception', class_weight=class_weight, params=params)

In [25]:
inception.construct_model()

==========================Downloading inceptionv3==========================
=============================Download Complete=============================
===============================Adding layers===============================
===============================Added layers================================
===========Saving weights to weight_path/inceptionv3_weights.h5============
===============================Saved weights===============================


In [17]:
xception.construct_model()

===========================Downloading xception============================
=============================Download Complete=============================
===============================Adding layers===============================
===============================Added layers================================
=============Saving weights to weight_path/xception_weights.h5=============
===============================Saved weights===============================


In [18]:
inception.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 19)                38931     
Total params: 21,841,715
Trainable params: 38,931
Non-trainable params: 21,802,784
_________________________________________________________________


In [19]:
xception.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 19)                38931     
Total params: 20,900,411
Trainable params: 38,931
Non-trainable params: 20,861,480
_________________________________________________________________


# 모델 훈련 
## Feature Extraction

In [ ]:
inception.train()
inception.save()

In [ ]:
xception.train()
xception.save()

## Fine Tuning

In [27]:
params.update({
    'mode': 'ft',
    'lr': 0.0001
})

In [ ]:
inception.train()
inception.save()

=======================training inceptionv3 with ft========================
Epoch 1/10
21043/21044 [============================>.] - ETA: 0s - loss: 1.0634 - categorical_accuracy: 0.6532 - top_3_accuracy: 0.8888

In [ ]:
xception.train()
xception.save()

## 모델 훈련 결과

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

headers = [
    'epoch',
    'categorical_accuracy',
    'loss',
    'top_3_accuracy',
    'val_categorical_accuracy',
    'val_loss',
    'val_top_3_accuracy'
]

inceptionv3_fe_log = 'log/inceptionv3_fe.log'
inceptionv3_ft_log = 'log/inceptionv3_ft.log'
xception_fe_log = 'log/xception_fe.log'
xception_ft_log = 'log/xception_ft.log'

logs = [inceptionv3_fe_log, inceptionv3_ft_log, xception_fe_log, xception_ft_log]

legends = [log.split('/')[1].split('.log')[0] for log in logs]

fig = plt.figure(figsize=(20,10))
for log in logs:
    df = pd.read_csv(log)
    # val loss subplot
    plt.subplot(2,2,1)
    plt.title('Validation Loss')
    plt.plot(df.epoch, df.val_loss)
    plt.legend(legends)
    
    # val accuracy subplot
    plt.subplot(2,2,2)
    plt.title('Validation Accuracy')
    plt.plot(df.epoch, df.val_categorical_accuracy)
    plt.legend(legends)
    
    # val top 3 accuracy subplot
    plt.subplot(2,2,3)
    plt.title('Validation Top 3 Accuracy')
    plt.plot(df.epoch, df.val_top_3_accuracy)
    plt.legend(legends)

plt.show()


# 모델 성능

## 모델 성능 helper functions

In [ ]:
def plot_images(images, cls_true, cls_pred):
    
    assert len(images) == len(cls_true)
    if len(images) == 0: return

    num_x_plot = int(np.ceil(len(images)/5))
    fig, axes = plt.subplots(num_x_plot , 5 , figsize=(22,len(images)))
    interpolation = 'spline16'

    fig.subplots_adjust(hspace=0.6)
  
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i], interpolation=interpolation)
            cls_pred_name = cls_pred.argsort(axis=1)[i][-3:][::-1]
            cls_pred_prob = np.sort(cls_pred, axis=1)[i][-3:][::-1]
            cls_true_name = cls_true[i]
            xlabel = ""
            for idx, class_name in enumerate(cls_pred_name):
                xlabel += "Pred {} : {}, score: {:.04f} \n".format(idx, idx_cls[class_name], cls_pred_prob[idx])
                xlabel += "True:{}".format(idx_cls[cls_true_name])
                ax.set_xlabel(xlabel, fontsize=15.0)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.tight_layout()
    plt.show()
    
def load_images(image_paths):
    images = [plt.imread(path) for path in image_paths]
    return np.asarray(images)

def print_errors(class_num, cls_test, cls_pred, size=15):
    generator_test.reset()

    cls_pred_argmax = cls_pred.argmax(axis=1) # 예측 클래스 어래이
    incorrect = (cls_pred_argmax != cls_test) # 틀린 예측 인덱스
    true_incorrect = cls_test[incorrect]
    pred_incorrect = cls_pred_argmax[incorrect]

    pred_idx = np.argwhere(pred_incorrect == class_num).flatten() 

    print('예측 틀린 개수:{}'.format(len(pred_idx)))

    rdm_idx = np.random.randint(0, len(pred_idx), size)
    # 틀린 index 중 주어진 카테고리 번호의 prediction 이 틀린 index 중 랜덤한 size개의 index의 image_path
    image_path = np.array(path_join(test_dir_name, generator_test.filenames))[incorrect][pred_idx][rdm_idx]
    image = load_images(image_path)

    true_incorrect = true_incorrect[pred_idx][rdm_idx]
    pred_incorrect = cls_pred[incorrect][pred_idx][rdm_idx]

    plot_images(images=image, cls_true=true_incorrect, cls_pred=pred_incorrect)
    
    
def print_confusion_matrix(cls_test, cls_pred, labels, labels_actual):
  
    cm = confusion_matrix(y_true=cls_test,
                          y_pred=cls_pred.argmax(axis=1),
                          labels=labels)
    
    true_pos = np.diag(cm)
    true_pos_false_pos = np.sum(cm, axis=0).astype(float)
    true_pos_false_neg = np.sum(cm, axis=1).astype(float)

    precision = true_pos / (true_pos_false_pos)
    recall = true_pos / (true_pos_false_neg)

    # please take care of divison by zero
    precision[np.isnan(precision)] = 0.0
    recall[np.isnan(recall)] = 0.0

    plt.figure(figsize=(20,10))
    sns.heatmap(cm, 
                annot=True, 
                xticklabels=label_actual, 
                yticklabels=label_actual, 
                fmt='g')

    plt.xlabel("predicted label")
    plt.ylabel("actual label")
    
    for i, class_name in enumerate(label_actual):
        print("({0}) {1}: recall={2:.2f}, precision={3:.2f}".format(i, class_name , recall[i], precision[i]))

In [ ]:
TEST_PATH = '../testing'
datagen_test = ImageDataGenerator(rescale=1./255)

batch_size = 256

generator_test = datagen_test.flow_from_directory(directory=TEST_PATH,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False
                                                  )

step_size = generator_test.n / batch_size + 1

# model로 test set 추론
xception_pred = xception.model.predict_generator(generator_test, steps=step_size, verbose=1)
inception_pred = inception.model.predict_generator(generator_test, steps=step_size, verbose=1)

classes = np.sort(generator_test.class_indices.values()) # 카테고리 번호 
classes_actual = np.sort(generator_test.class_indices.keys()) # 카테고리 명

cls_test = generator_test.classes

idx_cls = generator_test.class_indices()

In [ ]:
# Xception model fine tuning confusion matrix
print_confusion_matrix(cls_test, xception_pred, classes, classes_actual)

In [ ]:
# Inception model fine tuning confusion matrix
print_confusion_matrix(cls_test, inception_pred, classes, classes_actual)

In [ ]:
print_errors(0, cls_test, xception_pred)

In [ ]:
print_errors(10, cls_test, xception_pred)

In [ ]:
print_errors(6, cls_test, xception_pred)